In [1]:
import os

In [2]:
import time
import logging
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.profiler import profile, record_function, ProfilerActivity

In [4]:
def build_data_loader(data_dir, batch_size, random_seed=42, valid_size=0.1, shuffle=True, test=False):
    
    transform = transforms.Compose([transforms.Resize(255), 
                                    transforms.CenterCrop(224),  
                                    transforms.RandomHorizontalFlip(p=0.5),
                                    transforms.RandomRotation(20),
                                    transforms.GaussianBlur(3),
                                    transforms.ToTensor(), 
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]) 
    
    train_dataset = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=batch_size, 
                                               num_workers=8, 
                                               pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

    return (train_loader, test_loader)

In [5]:
def train(model, data_loader, num_epochs, criterion, optimizer, device):
    total_steps = len(train_loader)
    for epoch in range(num_epochs):
        start = time.time()
        for step, (images, labels) in enumerate(train_loader):  
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
        
            # Forward pass
            outputs = model(images)
            outputs.to(device)
            loss = criterion(outputs, labels)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print ('Step [{}/{}], Loss: {:.4f}'.format(step+1, total_steps, loss.item()))
        end = time.time()
               
        print ('Epoch [{}/{}], Loss: {:.4f}, time: {} seconds'.format(epoch+1, num_epochs, loss.item(), int(end-start)))
            

In [6]:
def test(model, test_loader, device):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))  

In [7]:
# General parameters
data_dir = '/tmp'
device = "cuda"

# Hyperparameters
lr = 0.00001
weight_decay = 0.005
batch_size = 128
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

# FashionMNIST dataset 
train_loader, test_loader = build_data_loader(data_dir=data_dir, batch_size=batch_size)

# Models
weights = models.DenseNet121_Weights.DEFAULT
net = models.densenet121(weights=weights)
model = net.train()
model = model.to(device)

# Optimizer
optimizer = optimizer(model.parameters(), lr, weight_decay=weight_decay)

#model, optimizer = ipex.optimize(model, optimizer=optimizer)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
start = time.time()
train(model, train_loader, num_epochs, criterion, optimizer, device)
end = time.time()
print('Training time: {} seconds'.format(int(end-start)))
test(model, test_loader, device)

Epoch [1/10], Loss: 1.3058, time: 97 seconds
Epoch [2/10], Loss: 0.7609, time: 82 seconds
Epoch [3/10], Loss: 0.5581, time: 83 seconds
Epoch [4/10], Loss: 0.3449, time: 82 seconds
Epoch [5/10], Loss: 0.1897, time: 81 seconds
Epoch [6/10], Loss: 0.2192, time: 82 seconds
Epoch [7/10], Loss: 0.1653, time: 83 seconds
Epoch [8/10], Loss: 0.1049, time: 82 seconds
Epoch [9/10], Loss: 0.1642, time: 82 seconds
Epoch [10/10], Loss: 0.0605, time: 82 seconds
Training time: 841 seconds
Accuracy of the network on the 10000 test images: 92.45 %
